In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Cell 2: Load and Inspect the Dataset
file_path = "ETB.csv"
data = pd.read_csv(file_path)

# Remove columns with 'Unnamed' in their name
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Display the first few rows of the cleaned dataset
data.head()

In [ ]:
# Cell 3: Preprocessing
data["Date"] = pd.to_datetime(data["Date"])
data["Days"] = (data["Date"] - data["Date"].min()).dt.days
data["Weight"] = np.where(data["Date"] >= pd.to_datetime("2020-01-01"), 8, 1)  # Increase weight to 5 for data after 2020
data_cleaned = data.dropna()
features = data_cleaned[["Days", "Open", "Low"]]
target = data_cleaned["High"]
weights = data_cleaned["Weight"]

data.head()

In [ ]:
# Cell 4: Split Data
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Cell 5: Hyperparameter Tuning using Gradient Descent
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Initialize alpha
alpha = 1.0

# Define learning rate and number of iterations
learning_rate = 0.01
num_iterations = 100

# Function to compute the gradient of the loss with respect to alpha
def compute_gradient(X, y, model, alpha):
    model.set_params(alpha=alpha)
    model.fit(X, y)
    y_pred = model.predict(X)
    gradient = -2 * np.sum((y - y_pred) * y_pred) / len(y)
    return gradient

# Gradient descent loop
for i in range(num_iterations):
    model = Ridge(alpha=alpha)
    gradient = compute_gradient(X_train, y_train, model, alpha)
    alpha -= learning_rate * gradient
    print(f"Iteration {i+1}: alpha = {alpha}")

# Train the final model with the optimized alpha
final_model = Ridge(alpha=alpha)
final_model.fit(X_train, y_train, sample_weight=weights.loc[X_train.index])

# Evaluate the final model on the test set
y_test_pred = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print(f"Optimized alpha: {alpha}")
print(f"Test Mean Squared Error: {mse}")
print(f"Test Mean Absolute Error: {mae}")
print(f"Test R-squared: {r2}")

In [ ]:
# Cell 5: Train the Model
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(X_train, y_train, sample_weight=weights.loc[X_train.index])
coefficients = pd.DataFrame({"Feature": features.columns, "Coefficient": model.coef_})
print(coefficients)
print(f"Intercept: {model.intercept_}")

In [ ]:
# Cell 6: Evaluate the Model
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# Cell 7: Visualization
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_test_pred, alpha=0.7, color="blue")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted High Values")
plt.show()

days_range = np.linspace(features["Days"].min(), features["Days"].max(), 100).reshape(-1, 1)
mean_values = np.mean(features[["Open", "Low"]], axis=0).values.reshape(1, -1)
mean_values_repeated = np.repeat(mean_values, days_range.shape[0], axis=0)
high_pred_line = model.predict(np.hstack([days_range, mean_values_repeated]))

# Plot the regression line
plt.figure(figsize=(10, 6))
plt.scatter(features["Days"], target, alpha=0.5, label="Actual Data")
plt.plot(days_range, high_pred_line, color="red", label="Regression Line")
plt.xlabel("Days Since Start")
plt.ylabel("High Price")
plt.title("Regression Line with Multiple Features")
plt.legend()
plt.show()


In [ ]:
# Cell 8: Prediction Interface
def predict_high(input_date):
    input_date = pd.to_datetime(input_date)
    days_since_start = (input_date - data_cleaned["Date"].min()).days
    closest_row = data_cleaned.iloc[(data_cleaned["Days"] - days_since_start).abs().argmin()]
    open_val = closest_row["Open"]
    low_val = closest_row["Low"]
    prediction = model.predict([[days_since_start, open_val, low_val]])[0]
    return prediction

input_date = input("Enter Date (YYYY-MM-DD): ")
predicted_high = predict_high(input_date)
print(f"Predicted High for {input_date}: {predicted_high}")